In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns

df = pd.read_csv("../input/heart-failure-prediction/heart.csv")

In [ ]:
df.head()

In [ ]:
sns.barplot(x=df.index, y=df['Age'])

In [ ]:
df.corr()['HeartDisease'].sort_values()

In [ ]:
print(df['ChestPainType'].unique())
print(df['RestingECG'].unique())
print(df['ST_Slope'].unique())

In [ ]:
df = pd.get_dummies(df)
df.head()

In [ ]:
X = df.drop('HeartDisease',axis = 1)
y = df['HeartDisease']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_x = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.17, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

lir = LinearRegression()

lir = lir.fit(x_train, y_train)

y_pred_lir = lir.predict(x_test)

print(lir.score(x_test,y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rgb = RandomForestRegressor()

rgb = rgb.fit(x_train, y_train)

y_pred_rgb = rgb.predict(x_test)

print(rgb.score(x_test,y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

hyper_params = {
    'max_depth':[3,4,5,6,7,8,9],
    'gamma':[0,0.5,1,0.09],
    'subsample':[0.6,0.7,0.8,0.9,1],
    'learning_rate':[0.01,0.1,0.2,0.3],
    'colsample_bytree':[0.5,0.6,0.7,0.8,0.9,1],
    'max_bin':[256,512,1024]
}

xgb = XGBRegressor(seed=13, tree_method = 'hist')

rcv = RandomizedSearchCV(
    estimator = xgb,
    param_distributions = hyper_params,
    n_iter = 4,
    scoring = 'neg_root_mean_squared_error',
    verbose = 2
)

rcv.fit(x_train, y_train)

print(rcv.best_params_)
print(rcv.best_score_)

In [ ]:
xgb = XGBRegressor(
    seed = 13,
    tree_method = 'hist',
    subsample = 0.6,
    max_depth = 6,
    max_bin = 256,
    learning_rate = 0.1,
    gamma=1,
    colsample_bytree = 0.5
)

xgb = xgb.fit(x_train, y_train)

y_test_xgb = xgb.predict(x_test)

In [ ]:
print(xgb.score(x_test, y_test))

In [ ]:
from sklearn.metrics import mean_absolute_error

xgb_score = mean_absolute_error(y_test_xgb, y_test)
print(xgb_score)

In [ ]:
rgb_score = mean_absolute_error(y_pred_rgb, y_test)
print(rgb_score)

In [ ]:
lir_score = mean_absolute_error(y_pred_lir, y_test)
print(lir_score)